In [ ]:
!pip install facenet-pytorch
!pip install Pillow

In [ ]:
from facenet_pytorch import InceptionResnetV1

resnet = InceptionResnetV1(pretrained='vggface2').eval()
resnet.classify = True


# test singole immagini

In [ ]:
import os
from io import BytesIO
from PIL import Image
import requests
from torchvision import transforms
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

def load_image(filename):
    file_path = os.path.join( "test_set_cropped", filename)
    img = Image.open(file_path)
    rsz = img.resize((160, 160))
    tns = transforms.ToTensor()(rsz)
    return tns, rsz

fpath = tf.keras.utils.get_file('rcmalli_vggface_labels_v2.npy',
                             "https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_labels_v2.npy",
                             cache_subdir="./")
LABELS = np.load(fpath)

# Brad Pitt
brad_img, brad_image = load_image("Brad_Pitt_2_face_0.jpg")

# Vasco
cr7_img, cr7_image = load_image("Vasco_Rossi_10_face_0.jpg")

brad_probs = resnet(brad_img.unsqueeze(0))
brad_pred = LABELS[np.array(brad_probs[0].detach().numpy()).argmax()]
print(brad_pred)

cr7_probs = resnet(cr7_img.unsqueeze(0))
cr7_pred = LABELS[np.array(cr7_probs[0].detach().numpy()).argmax()]
print(cr7_pred)

In [ ]:
plt.figure()
plt.matshow(brad_image)
plt.title("Model Prediction: {}".format(brad_pred))
plt.show()

plt.figure()
plt.matshow(cr7_image)
plt.title("Model Prediction: {}".format(cr7_pred))
plt.show()

# TESTING NN1 on test set clean

In [ ]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf
import re
from torchvision import transforms


def load_image(filename):
    img = Image.open(filename)
    rsz = img.resize((160, 160))
    tns = transforms.ToTensor()(rsz)
    return tns


fpath = tf.keras.utils.get_file('rcmalli_vggface_labels_v2.npy',
                             "https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_labels_v2.npy",
                             cache_subdir="./")
LABELS = np.load(fpath)

dataset_dir = "test_set_cropped"

correct_predictions = 0
total_images = 0

for filename in os.listdir(dataset_dir):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        person_path = os.path.join(dataset_dir, filename)
        img = load_image(person_path)
        probs = resnet(img.unsqueeze(0))
        pred_label = LABELS[np.array(probs.detach().numpy()).argmax()]
        print("Immagine:", filename)
        print("Predizione del modello:", pred_label)
        correct_label = re.sub(r'_\d+_face_0\.jpg$', '', filename)
        print("Etichetta corretta:", correct_label)


        if correct_label in pred_label:
                correct_predictions += 1
                print("OK")
        else:
             with open('predizioni_errate.txt', 'a') as file:
              file.write(filename)
              file.write('\n')

        total_images += 1
        print("-------------------------->")


if total_images == 0:
    print("Nessuna immagine trovata nel dataset.")
else:
    test_accuracy = correct_predictions / total_images
    print('\n')
    print('\n')
    print("Test Accuracy:", test_accuracy)
    print("Numero di predizioni corrette:", correct_predictions)
    print("Numero di immagini processate:", total_images)